In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Cargar el archivo limpio de cáncer de mama
df = pd.read_csv('cancer_de_mama_limpio.csv')

# Extraer variables de entrada (todas las filas, columnas de 2 a 31)
X = df.iloc[:, 2:31].values

# Extraer columna de salida (todas las filas, columna 'diagnosis')
Y = df.iloc[:, 1].values

# Normalizar las características
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Separar los datos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3, random_state=42)

# Crear la red neuronal
nn = MLPClassifier(solver='adam',
                   hidden_layer_sizes=(100,100, ),  # Capas ocultas
                   activation='relu',  # Función de activación
                   max_iter=75000,  # Máximo de iteraciones
                   learning_rate_init=0.01)  # Tasa de aprendizaje inicial

# Entrenamiento
nn.fit(X_train, Y_train)

# Evaluación del modelo
print("Porcentaje de aciertos con train: ", (nn.score(X_train, Y_train) * 100))
print("Porcentaje de aciertos con test: ", (nn.score(X_test, Y_test) * 100))

Porcentaje de aciertos con train:  100.0
Porcentaje de aciertos con test:  96.84210526315789


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar el archivo limpio de cáncer de mama
df2 = pd.read_csv('cancer_de_mama_limpio.csv')

# Extraer variables de entrada (todas las filas, columnas de 2 a 31)
X = df2.iloc[:, 2:].values

# Extraer columna de salida (todas las filas, columna 'diagnosis')
Y = df2.iloc[:, 1].values  

# Normalizar las características con StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3)

# Número de registros de entrenamiento
n = X_train.shape[0]
print('Cantidad de filas de entrenamiento: ', n)

# Construir red neuronal con pesos y sesgos inicializados aleatoriamente
leaky_relu = lambda x: np.where(x > 0, x, x * 0.01)
logistic = lambda x: 1 / (1 + np.exp(-x))

# Semilla para reproducibilidad
np.random.seed(2)

# Inicialización de pesos
w_hidden = (np.random.rand(4, 30)*2)-1      
w_hidden2 = (np.random.rand(4, 4)*2)-1   
w_output = (np.random.rand(1, 4)*2)-1         

# Inicialización de sesgos
b_hidden = (np.random.rand(4, 1)*2)-1         
b_hidden2 = (np.random.rand(4, 1)*2)-1        
b_output = (np.random.rand(1, 1)*2)-1

# Ejecutar entradas a través de la red neuronal para obtener salidas predichas
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden        
    A1 = leaky_relu(Z1)
    Z2 = w_hidden2 @ A1 + b_hidden2     
    A2 = leaky_relu(Z2)
    Z3 = w_output @ A2 + b_output       
    A3 = logistic(Z3)
    return Z1, A1, Z2, A2, Z3, A3

# Calculo de precisión
def precision(X, Y):
    # Llama a la función de propagación hacia adelante (forward_prop) usando los datos de entrada X y toma la salida predicha 
    # que es el valor en la posición [5]
    test_predictions = forward_prop(X.transpose())[5]
    
    # Convierte las predicciones en 1 o 0 según si son mayores o menores a 0.5, y las compara con los valores reales (Y)
    test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y)
    
    # Calcula la precisión como la cantidad de aciertos sobre el total de ejemplos.
    accuracy = sum(test_comparisons.astype(int) / X.shape[0])
    
    # Imprime el porcentaje de precisión, redondeado a dos decimales.
    print("Porcentaje de aciertos: ", (accuracy*100).round(2))

print('Pre entrenamiento: \n')
print('Test')
precision(X_test, Y_test)
print('Train')
precision(X_train, Y_train)

# Se realiza el descenso de gradiente y calculo la red entrenada

# Tasa de aprendizaje
L = 0.01

# Derivadas de las funciones de activación
d_leaky_relu = lambda x: np.where(x > 0, x, 0.01) 
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Función de backward propagation
def backward_prop(Z1, A1, Z2, A2, Z3, A3, X, Y):
    dC_dA3 = 2 * (A3 - Y)           
    dA3_dZ3 = d_logistic(Z3)
    dZ3_dW3 = A2
    dZ3_dA2 = w_output
    dC_dZ3 = dC_dA3 * dA3_dZ3

    dC_dA2 = dZ3_dA2.T @ dC_dZ3
    dA2_dZ2 = d_leaky_relu(Z2)
    dZ2_dW2 = A1
    dZ2_dA1 = w_hidden2
    dC_dZ2 = dC_dA2 * dA2_dZ2

    dC_dA1 = dZ2_dA1.T @ dC_dZ2
    dA1_dZ1 = d_leaky_relu(Z1)
    dZ1_dW1 = X
    dC_dZ1 = dC_dA1 * dA1_dZ1

    dC_dW3 = dC_dZ3 @ dZ3_dW3.T
    dC_dB3 = np.sum(dC_dZ3, axis=1, keepdims=True)

    dC_dW2 = dC_dZ2 @ dZ2_dW2.T
    dC_dB2 = np.sum(dC_dZ2, axis=1, keepdims=True)

    dC_dW1 = dC_dZ1 @ dZ1_dW1.T
    dC_dB1 = np.sum(dC_dZ1, axis=1, keepdims=True)
    
    return dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3

for i in range(75000):
    # seleccionar aleatoriamente uno de los datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2, Z3, A3 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación
    # y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2, dW3, dB3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, X_sample, Y_sample)

    # actualizar pesos y sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_hidden2 -= L * dW2
    b_hidden2 -= L * dB2
    w_output -= L * dW3
    b_output -= L * dB3

# Cálculo de precisión
print('Post entrenamiento: \n')
print('Test')
precision(X_test, Y_test)
print('Train')
precision(X_train, Y_train)

Cantidad de filas de entrenamiento:  379
Pre entrenamiento: 

Test
Porcentaje de aciertos:  63.16
Train
Porcentaje de aciertos:  63.32
Post entrenamiento: 

Test
Porcentaje de aciertos:  97.89
Train
Porcentaje de aciertos:  99.21
